In [77]:
import pandas as pd
import logging
import psycopg2
from sqlalchemy import create_engine
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')

# Define the PostgreSQL connection parameters
db_params = {
    'dbname': "InvoiceDB",
    'user': "postgres",
    'password': 12345,
    'host': 'localhost',  # Replace with your host
    'port': '5432',  # Replace with your port
}

# Establish a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')
table_names = ["DimClient", "DimDate", "DimMeter", "DimPayment"]

def get_data_from_postgresql(engine, table_names):
    dataframes = {}
    for table_name in table_names:
        query = f'SELECT * FROM "{table_name}"'
        dataframes[table_name] = pd.read_sql(query, engine)
    return dataframes

data = get_data_from_postgresql(engine, table_names)

DimClient = data["DimClient"]
DimDate = data["DimDate"]
DimMeter = data["DimMeter"]
DimPayment = data["DimPayment"]

# display(DimPayment)
# display(DimDate)

def create_fact_table(DimMeter, DimClient, DimPayment, DimDate):
    FactInvoice = pd.merge(DimMeter, DimClient, on="MeterID", how= "inner")
    FactInvoice['From'] = FactInvoice['Date'] - pd.offsets.MonthBegin(1)
    FactInvoice['Till'] = FactInvoice['Date'] + pd.offsets.MonthEnd(0)
    FactInvoice['OutgoingBill'] = FactInvoice['Till'] + pd.offsets.MonthBegin(1)
    FactInvoice['PaymentDeadline'] = FactInvoice['OutgoingBill'] + pd.Timedelta(days=60)
    FactInvoice['KwhUsage'] = FactInvoice['Last_Meter_Reading'] - FactInvoice['Initial_Meter_Reading']
    # FactInvoice = pd.merge(DimDate, FactInvoice,  on="Date", how= "inner")
    # DimPayment['Date'] = pd.to_datetime(DimPayment['Date'])
    # FactInvoice = pd.merge(FactInvoice, DimPayment,  on=["Date","ClientID"], how='outer')
    display(FactInvoice)
    logging.info(' FactInvoice data successfully created')
    return FactInvoice

def load(FactInvoice):
    FactInvoice.to_sql('FactInvoice', engine, if_exists='replace', index=False)

if __name__ == '__main__':
    FactInvoice = create_fact_table(DimMeter, DimClient, DimPayment, DimDate)
    load(FactInvoice)


,Date,MeterID,Initial_Meter_Reading,Last_Meter_Reading,ClientID,Name,Address,Rates,From,Till,OutgoingBill,PaymentDeadline,KwhUsage
0,2022-06-01,84691914,218845.000,344125.000,8,AEON CO. (M) BHD,"2,109 kWp Rooftop Solar Facility at AEON CO. (...",0.2820,2022-05-01,2022-06-30,2022-07-01,2022-08-30,125280.000
1,2022-07-01,84691914,344125.000,471199.000,8,AEON CO. (M) BHD,"2,109 kWp Rooftop Solar Facility at AEON CO. (...",0.2820,2022-06-01,2022-07-31,2022-08-01,2022-09-30,127074.000
2,2022-08-01,84691914,471199.000,614020.000,8,AEON CO. (M) BHD,"2,109 kWp Rooftop Solar Facility at AEON CO. (...",0.2820,2022-07-01,2022-08-31,2022-09-01,2022-10-31,142821.000
3,2022-09-01,84691914,614020.000,745816.384,8,AEON CO. (M) BHD,"2,109 kWp Rooftop Solar Facility at AEON CO. (...",0.2820,2022-08-01,2022-09-30,2022-10-01,2022-11-30,131796.384
4,2022-10-01,84691914,745816.384,883899.000,8,AEON CO. (M) BHD,"2,109 kWp Rooftop Solar Facility at AEON CO. (...",0.2820,2022-09-01,2022-10-31,2022-11-01,2022-12-31,138082.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2023-02-01,84491051,1832888.000,1971226.000,6,HICOM-Teck See Manufacturing Sdn Bhd,"2,000 kWp Rooftop Solar Facility at HICOM-Teck...",0.1898,2023-01-01,2023-02-28,2023-03-01,2023-04-30,138338.000
62,2023-03-01,84491051,1971226.000,2135219.000,6,HICOM-Teck See Manufacturing Sdn Bhd,"2,000 kWp Rooftop Solar Facility at HICOM-Teck...",0.1898,2023-02-01,2023-03-31,2023-04-01,2023-05-31,163993.000
63,2023-04-01,84491051,2135219.000,2247987.000,6,HICOM-Teck See Manufacturing Sdn Bhd,"2,000 kWp Rooftop Solar Facility at HICOM-Teck...",0.1898,2023-03-01,2023-04-30,2023-05-01,2023-06-30,112768.000
64,2023-05-01,84491051,2247987.000,2435238.000,6,HICOM-Teck See Manufacturing Sdn Bhd,"2,000 kWp Rooftop Solar Facility at HICOM-Teck...",0.1898,2023-04-01,2023-05-31,2023-06-01,2023-07-31,187251.000


2023-10-22 13:18:17,164:create_fact_table:INFO: FactInvoice data successfully created
